In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import difflib

In [2]:
#import box score data
data = pd.read_csv('C:\\Users\gsteele\gamelogsABA.csv')
df = pd.DataFrame(data = data)
df.insert(loc = 0,column = 'game_id', value = df['M'].astype(str)+df['D'].astype(str)
          +df['Season'].astype(str).str.slice(1,4)+df['Season'].astype(str).str.slice(6,8)
          +df['team_1'].astype(str)+df['team_2'].astype(str))
df.insert(loc = 3,column = 'Name',value = df['first_name']+' '+df['last_name'])
df.insert(loc = 1,column = 'playerid',value = df['first_name']+df['last_name']+(df['Season']).astype(str))

#Clean NaNs
df['Rookie'] = df['Rookie'].fillna(value = 'No')

In [3]:
print(df)

               game_id                 playerid first_name   last_name  \
0      101319668ANAOAK       BobBedell(1967-68)        Bob      Bedell   
1      101319668ANAOAK      LarryBunce(1967-68)      Larry       Bunce   
2      101319668ANAOAK   StephenChubin(1967-68)    Stephen      Chubin   
3      101319668ANAOAK  JeffreyCongdon(1967-68)    Jeffrey     Congdon   
4      101319668ANAOAK     HarryDinnel(1967-68)      Harry      Dinnel   
...                ...                      ...        ...         ...   
76807   51319776NYNDEN      TedMcClain(1975-76)        Ted     McClain   
76808   51319776NYNDEN  BillMelchionni(1975-76)       Bill  Melchionni   
76809   51319776NYNDEN       AlSkinner(1975-76)         Al     Skinner   
76810   51319776NYNDEN     BrianTaylor(1975-76)      Brian      Taylor   
76811   51319776NYNDEN  JohnWilliamson(1975-76)       John  Williamson   

                  Name position  Rookie Type   D   M  ...  OFF  DEF  REB  AST  \
0           Bob Bedell        

In [4]:
#import season totals, averages
season_data = pd.read_csv('C:\\Users\gsteele\ABAseasonstats.csv',low_memory = False)
season_frame = pd.DataFrame(data = season_data)
season_frame.set_index('League')
season = season_frame[season_frame.League == 'ABA']
season.insert(loc = 1,column  = 'playerid', 
value = season['firstname']+season['lastname']+season['Season'].astype(str))
seasons = season.fillna(value = 0.0)

In [5]:
def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.9):
    df_other= seasons.copy()
    df_other[left_on] = [get_closest_match(x, df[left_on], cutoff) 
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else x

In [6]:
aba = fuzzy_merge(df,seasons,left_on = 'playerid',right_on = 'playerid')
print(len(df))
print(len(aba))

76812
76810


In [7]:
print(aba.columns)

Index(['game_id', 'playerid', 'first_name', 'last_name', 'Name', 'position',
       'Rookie', 'Type', 'D', 'M', 'Y', 'Season_x', 'team_1', 'team_2',
       'Score', 'Unnamed: 12', 'W/L', 'OT', 'G_x', 'GS_x', 'MIN', 'FGM', 'FGA',
       'FG_Percentage', '3FGM', '3FGA', '3FG_Percentage', 'FTM', 'FTA',
       'FT_Percentage', 'OFF', 'DEF', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF',
       'PTS', 'Notes', 'League', 'firstname', 'lastname', 'pos', 'Rook ',
       'Champion', 'season_type', 'Season_y', 'Team(s)', 'Games', 'Started',
       'MP', 'fieldgoalsmad', 'fieldgoalsatt', 'fieldgoalpercentage',
       'threesmade', 'threesatt', 'threepointpercentage', 'freethrowsmade',
       'freethrowsatt', 'freethrowpercentage', 'offreb', 'defreb',
       'totalrebounds', 'assists', 'steals', 'blocks', 'turnovers', 'fouls',
       'points', 'Unnamed: 30', 'G_y', 'GS_y', 'mpg', 'fgmpg', 'fgapg', 'FG%',
       'fg3mpg', 'fg3apg', '3FG%', 'ftmpg', 'ftapm', 'FT%', 'orebpg', 'drebpg',
       'rebpg', 'apg'

In [8]:
#Calculating sums from game records

aba.insert(loc = 92, column = 'sum_pts', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['PTS'].transform("sum")))
aba.insert(loc = 93, column = 'sum_fgm', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['FGM'].transform("sum")))
aba.insert(loc = 94, column = 'sum_fga', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['FGA'].transform("sum")))
aba.insert(loc = 95, column = 'sum_fg3m', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['3FGM'].transform("sum")))
aba.insert(loc = 96, column = 'sum_fg3a', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['3FGA'].transform("sum")))
aba.insert(loc = 97, column = 'sum_ftm', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['FTM'].transform("sum")))
aba.insert(loc = 98, column = 'sum_fta', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['FTA'].transform("sum")))
aba.insert(loc = 99, column = 'sum_oreb', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['OFF'].transform("sum")))
aba.insert(loc = 100, column = 'sum_dreb', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['DEF'].transform("sum")))
aba.insert(loc = 101, column = 'sum_reb', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['REB'].transform("sum")))
aba.insert(loc = 102, column = 'sum_ast', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['AST'].transform("sum")))
aba.insert(loc = 103, column = 'sum_tov', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['TO'].transform("sum")))
aba.insert(loc = 104, column = 'sum_stl', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['STL'].transform("sum")))
aba.insert(loc = 105, column = 'sum_blk', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['BLK'].transform("sum")))
aba.insert(loc = 106, column = 'sum_pf', value = (aba[(aba['Type'] == 'Reg')].groupby(['Season_x','playerid'])['PF'].transform("sum")))

In [9]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [10]:
aba.sort_values(by = ['sum_pts'], ascending = False)

,game_id,playerid,first_name,last_name,Name,position,Rookie,Type,D,M,Y,Season_x,team_1,team_2,Score,Unnamed: 12,W/L,OT,G_x,GS_x,MIN,FGM,FGA,FG_Percentage,3FGM,3FGA,3FG_Percentage,FTM,FTA,FT_Percentage,OFF,DEF,REB,AST,STL,BLK,TO,PF,PTS,Notes,League,firstname,lastname,pos,Rook,Champion,season_type,Season_y,Team(s),Games,Started,MP,fieldgoalsmad,fieldgoalsatt,fieldgoalpercentage,threesmade,threesatt,threepointpercentage,freethrowsmade,freethrowsatt,freethrowpercentage,offreb,defreb,totalrebounds,assists,steals,blocks,turnovers,fouls,points,Unnamed: 30,G_y,GS_y,mpg,fgmpg,fgapg,FG%,fg3mpg,fg3apg,3FG%,ftmpg,ftapm,FT%,orebpg,drebpg,rebpg,apg,spg,bpg,tovpg,fpg,ppg,sum_pts,sum_fgm,sum_fga,sum_fg3m,sum_fg3a,sum_ftm,sum_fta,sum_oreb,sum_dreb,sum_reb,sum_ast,sum_tov,sum_stl,sum_blk,sum_pf
43267,102019772KENMEM,DanIssel(1971-72),Dan,Issel,Dan Issel,F-C,No,Reg,20,10,1971,(1971-72),KEN,MEM,106,104,W,NaN,1,1.0,NaN,13,26.0,50%,0,0.0,NaN,5,8,63%,NaN,NaN,8.0,NaN,NaN,NaN,NaN,4.0,31,NaN,ABA,Dan,Issel,C-F,0,0,Regular Season,(1971-72),KEN,83,,"3,570",972,"2,001",49%,3,11,27%,591,753,78%,353,578,931,195,,,244,242,"2,538",0.0,,,43.01,11.71,24.11,49%,0.04,0.13,27%,7.12,9.07,78%,4.25,6.96,11.22,2.35,,,2.94,2.92,30.58,2538.0,972.0,1983.0,3.0,11.0,591.0,753.0,0.0,0.0,931.0,152.0,168.0,0.0,0.0,239.0
43274,11219772KENMEM,DanIssel(1971-72),Dan,Issel,Dan Issel,F-C,No,Reg,2,11,1971,(1971-72),KEN,MEM,123,109,W,NaN,1,NaN,NaN,18,33.0,55%,0,0.0,NaN,3,5,60%,NaN,NaN,12.0,NaN,NaN,NaN,NaN,2.0,39,NaN,ABA,Dan,Issel,C-F,0,0,Regular Season,(1971-72),KEN,83,,"3,570",972,"2,001",49%,3,11,27%,591,753,78%,353,578,931,195,,,244,242,"2,538",0.0,,,43.01,11.71,24.11,49%,0.04,0.13,27%,7.12,9.07,78%,4.25,6.96,11.22,2.35,,,2.94,2.92,30.58,2538.0,972.0,1983.0,3.0,11.0,591.0,753.0,0.0,0.0,931.0,152.0,168.0,0.0,0.0,239.0
43339,31719772KENVIR,DanIssel(1971-72),Dan,Issel,Dan Issel,F-C,No,Reg,17,3,1972,(1971-72),KEN,VIR,112,104,W,NaN,1,1.0,46:00,10,26.0,38%,0,0.0,NaN,10,10,100%,NaN,NaN,19.0,0.0,NaN,NaN,3.0,2.0,30,NaN,ABA,Dan,Issel,C-F,0,0,Regular Season,(1971-72),KEN,83,,"3,570",972,"2,001",49%,3,11,27%,591,753,78%,353,578,931,195,,,244,242,"2,538",0.0,,,43.01,11.71,24.11,49%,0.04,0.13,27%,7.12,9.07,78%,4.25,6.96,11.22,2.35,,,2.94,2.92,30.58,2538.0,972.0,1983.0,3.0,11.0,591.0,753.0,0.0,0.0,931.0,152.0,168.0,0.0,0.0,239.0
43281,111719772KENVIR,DanIssel(1971-72),Dan,Issel,Dan Issel,F-C,No,Reg,17,11,1971,(1971-72),KEN,VIR,117,112,W,NaN,1,1.0,NaN,8,19.0,42%,0,0.0,NaN,9,13,69%,NaN,NaN,10.0,NaN,NaN,NaN,NaN,3.0,25,NaN,ABA,Dan,Issel,C-F,0,0,Regular Season,(1971-72),KEN,83,,"3,570",972,"2,001",49%,3,11,27%,591,753,78%,353,578,931,195,,,244,242,"2,538",0.0,,,43.01,11.71,24.11,49%,0.04,0.13,27%,7.12,9.07,78%,4.25,6.96,11.22,2.35,,,2.94,2.92,30.58,2538.0,972.0,1983.0,3.0,11.0,591.0,753.0,0.0,0.0,931.0,152.0,168.0,0.0,0.0,239.0
43280,111319772KENIND,DanIssel(1971-72),Dan,Issel,Dan Issel,F-C,No,Reg,13,11,1971,(1971-72),KEN,IND,111,116,L,NaN,1,NaN,44:00,15,35.0,43%,0,0.0,NaN,2,4,50%,NaN,NaN,17.0,2.0,NaN,NaN,NaN,6.0,32,NaN,ABA,Dan,Issel,C-F,0,0,Regular Season,(1971-72),KEN,83,,"3,570",972,"2,001",49%,3,11,27%,591,753,78%,353,578,931,195,,,244,242,"2,538",0.0,,,43.01,11.71,24.11,49%,0.04,0.13,27%,7.12,9.07,78%,4.25,6.96,11.22,2.35,,,2.94,2.92,30.58,2538.0,972.0,1983.0,3.0,11.0,591.0,753.0,0.0,0.0,931.0,152.0,168.0,0.0,0.0,239.0
43279,111019772KENDAL,DanIssel(1971-72),Dan,Issel,Dan Issel,F-C,No,Reg,10,11,1971,(1971-72),KEN,DAL,115,89,W,NaN,1,1.0,NaN,8,20.0,40%,0,0.0,NaN,10,15,67%,NaN,NaN,6.0,NaN,NaN,NaN,NaN,1.0,26,NaN,ABA,Dan,Issel,C-F,0,0,Regular Season,(1971-72),KEN,83,,"3,570",972,"2,001",49%,3,11,27%,591,753,78%,353,578,931,195,,,244,242,"2,538",0.0,,,43.01,11.71,24.11,49%,0.04,0.13,27%,7.12,9.07,78%,4.25,6.96,11.22,2.35,,,2.94,2.92,30.58,2538.0,972.0,1983.0,3.0,11.0,591.0,753.0,0.0,0.0,931.0,152.0,168.0,0.0,0.0,239.0
43278,11919772KENDEN,DanIssel(1971-72),Dan,Issel,Dan Issel,F-C,No,Reg,9,11,1971,(1971-72),KEN,DEN,106,96,W,NaN,1,1.0,NaN,8,21.0,38%,0,0.0,NaN,6,8,75%,NaN,NaN,13.0,NaN,NaN,NaN,NaN,2.0,22,N

In [11]:
len(aba[(aba['points'] == 0) & (aba['sum_pts'] != 0)])

0

In [12]:
aba.insert(loc = 107, column = 'res_pts', 
            value = (aba['points'].str.replace(',','').astype(float) 
               - aba['sum_pts']))
aba.insert(loc = 107, column = 'res_fgm', 
            value = (aba['fieldgoalsmad'].str.replace(',','').astype(float)
                - aba['sum_fgm']))
aba.insert(loc = 107, column = 'res_fga', 
           value = (aba['fieldgoalsatt'].str.replace(',','').astype(float)
                - aba['sum_fga']))
aba.insert(loc = 107, column = 'res_fg3m', 
           value = (aba['threesmade'].str.replace(',','').astype(float)
                - aba['sum_fg3m']))
aba.insert(loc = 107, column = 'res_fg3a', 
           value = (aba['threesatt'].str.replace(',','').astype(float)
                - aba['sum_fg3a']))
aba.insert(loc = 107, column = 'res_ftm', 
           value = (aba['freethrowsmade'].astype(float)
                - aba['sum_ftm']))
aba.insert(loc = 107, column = 'res_fta', 
           value = (aba['freethrowsatt'].str.replace(',','').astype(float)
                - aba['sum_fta']))
aba.insert(loc = 107, column = 'res_oreb', 
           value = (aba['offreb'].str.replace(',','').astype(float)
                - aba['sum_oreb'])) 
aba.insert(loc = 107, column = 'res_dreb', 
           value = (aba['defreb'].str.replace(',','').astype(float)
                - aba['sum_dreb']))
aba.insert(loc = 107, column = 'res_reb', 
           value = (aba['totalrebounds'].str.replace(',','').astype(float)
                - aba['sum_reb']))
aba.insert(loc = 107, column = 'res_ast', 
           value = (aba['assists'].str.replace(',','').astype(float)
                - aba['sum_ast']))
aba.insert(loc = 107, column = 'res_tov', 
           value = (aba['turnovers'].str.replace(',','').astype(float)
                - aba['sum_tov']))
aba.insert(loc = 107, column = 'res_pf', 
           value = (aba['fouls'].str.replace(',','').astype(float)
                - aba['sum_pf']))

#Cannot convert empty cells to float
#aba.insert(loc = 107, column = 'res_stl', value = (aba['sum_stl'] - aba['steals'].str.replace(',','').astype(float)))
#aba.insert(loc = 107, column = 'res_blk', value = (aba['sum_blk'] - aba['blocks'].str.replace(',','').astype(float)))



In [20]:
#aba['steals'] = aba['steals'].str.strip()
aba['steals2'] = aba['steals'].str.replace(' ','.0')
aba['steals2'].value_counts()

.0     44965
59       649
47       552
40       530
60       503
57       456
48       423
63       402
64       374
89       357
37       349
65       343
27       338
16       333
67       331
28       323
31       306
36       301
44       285
166      282
52       279
115      277
26       269
154      268
56       268
69       267
76       265
61       264
96       259
92       259
105      257
9        243
46       240
32       238
45       237
24       214
53       204
51       203
120      192
167      191
107      189
12       189
123      187
84       185
23       185
91       185
85       183
131      183
70       182
79       181
72       181
170      179
108      178
136      175
100      174
13       173
206      173
81       171
54       168
129      168
88       165
50       165
4        161
127      160
82       160
38       157
33       156
73       156
55       155
103      147
11       143
97       140
68       139
14       138
71       135
62       133
29       133

In [22]:
aba.insert(loc = 107, column = 'res_stl', value = (aba['sum_stl'] - aba['steals'].astype(float)))

ValueError: could not convert string to float: 